In [9]:
# moderator_app.py
from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Literal

from dotenv import load_dotenv
load_dotenv()  # loads OPENAI_API_KEY from .env

from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

# -----------------------------
# 1) STATE (shared memory)
# -----------------------------
@dataclass
class DebateCitation:
    title: str
    url: str
    snippet: Optional[str] = None

@dataclass
class DebateMessage:
    content: str
    key_points: List[str] =  field(default_factory=list)
    citations: List[DebateCitation] = field(default_factory=list)

@dataclass
class JudgeMessage:
    decision: Literal["pro", "con", "tie"]
    rationale: str
    pro_feedback: str
    con_feedback: str
    key_points: List[str] =  field(default_factory=list)

@dataclass
class DebateHistory:
    role: Literal["pro", "con", "judge"]
    phase: Literal["opening", "rebuttal", "closing", "decision"]
    output: DebateMessage

@dataclass
class DebateState:
    topic: str
    status: str = "init"                       # "init" | "rejected" | "ready"
    safety_flags: List[Dict] = field(default_factory=list)  # audit trail
    message: Optional[str] = None              # explanation / next-step text
    history: List[DebateHistory] = field(default_factory=list)  # past messages


# -----------------------------
# 2) SCHEMA for structured output
# -----------------------------
Action = Literal["allow", "rephrase", "reject"]

class ModerationResult(BaseModel):
    action: Action
    categories: List[str] = Field(default_factory=list)     # e.g., ["illegal_activity", "hate"]
    explanation: str                                        # short human-safe rationale
    safe_topic: Optional[str] = None                        # required if action == "rephrase"
    notes: List[str] = Field(default_factory=list)          # optional extra guidance

# Use a Pydantic model for structured LLM output, then map to dataclass
class DebateCitationModel(BaseModel):
    title: str
    url: str
    snippet: Optional[str] = None

class DebateMessageModel(BaseModel):
    content: str = Field(..., max_length=2000)  # Limit content length
    key_points: List[str] = Field(default_factory=list, max_length=3)  # Limit key points
    citations: List[DebateCitationModel] = Field(default_factory=list, max_length=2)  # Limit citations

class JudgeMessageModel(BaseModel):
    decision: Literal["pro", "con", "tie"]
    rationale: str = Field(..., max_length=300)  # Limit rationale length
    pro_feedback: str = Field(..., max_length=200)
    con_feedback: str = Field(..., max_length=200)
    key_points: List[str] = Field(default_factory=list, max_length=3)


# -----------------------------
# 3) LLM setup with token limits
# -----------------------------
llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0,
    max_tokens=2000,  # Set a reasonable max_tokens limit
    request_timeout=30  # seconds
)

# -----------------------------
# 4) NODES
# -----------------------------
def moderator_node(state: DebateState) -> DebateState:
    """Classify a topic as allow / rephrase / reject; optionally rephrase it."""
    # 4a) quick keyword short-circuit (fast, local)
    quick_block = [
        "build a bomb", "make a bomb", "child sexual", "sexual content with minors",
        "self-harm instructions", "how to hack", "credit card skimming",
        "assassinate", "terrorist", "make a gun", "kill"
    ]
    topic_l = (state.topic or "").strip().lower()
    if any(k in topic_l for k in quick_block):
        res = ModerationResult(
            action="reject",
            categories=["illegal_or_extreme_harm"],
            explanation="Disallowed: violent/illegal instruction content.",
            safe_topic=None,
            notes=["Reframe as an ethics/policy discussion rather than instructions."]
        )
        state.safety_flags.append(res.model_dump())
        return state

    # 4b) LLM moderation + academic reframe
    system = (
        "You are a safety moderator for a debate platform. "
        "Decide one of: allow | rephrase | reject. "
        "Disallow topics with step-by-step illegal harm, sexual content with minors, "
        "self-harm instructions, targeted hate/violence, invasive privacy breaches, malware creation, etc. "
        "Also don't allow topics that have any relation to religion or politics, the topics should all be "
        "leaning more towards being lighthearted and less controversial. "
        "If rephrase: produce a neutral, academic framing (policy/ethics/history) that preserves learning value. "
        "Also ensure that the rephrase is detailed and provides two clear sides to argue about. "
        "Keep your response concise. Return ONLY structured JSON matching the provided schema."
    )
    user = f'Topic: "{state.topic}"'

    mod_llm = llm.with_structured_output(schema=ModerationResult)
    result: ModerationResult = mod_llm.invoke(f"{system}\n\n{user}")

    # audit trail
    state.safety_flags.append(result.model_dump())

    # apply rephrase if given
    if result.action == "rephrase" and result.safe_topic:
        state.topic = result.safe_topic

    return state

def moderator_router(state: DebateState) -> str:
    """Decide which node to run after moderator."""
    last = state.safety_flags[-1] if state.safety_flags else {}
    action = last.get("action")
    if action == "reject":
        return "reject_exit"
    return "proceed"  # allow OR rephrase both continue

def reject_exit(state: DebateState) -> DebateState:
    state.status = "rejected"
    last = state.safety_flags[-1] if state.safety_flags else {}
    state.message = f"❌ Rejected. Reason: {last.get('explanation','')}"
    return state

def proceed_node(state: DebateState) -> DebateState:
    state.status = "ready"
    last = state.safety_flags[-1] if state.safety_flags else {}
    if last.get("action") == "rephrase":
        state.message = f"✅ Rephrased to safe topic: {state.topic}. Ready to start the debate flow."
    else:
        state.message = f"✅ Allowed topic: {state.topic}. Ready to start the debate flow."
    return state


# ---------- PRO / CON NODES ----------
def pro_node(state: DebateState) -> DebateState:
    """Generate Pro opening or rebuttal, then store into history."""
    system = (
        "You are the 'Pro' debater in a formal debate. "
        "Support the topic with clear reasoning and evidence. "
        "Be concise and persuasive. Keep your response between 150-200 words. "
        # "Focus on 2-3 key points maximum. Provide 1-2 citations if relevant."
    )
    mod_llm = llm.with_structured_output(schema=DebateMessageModel)

    if not state.history:
        # Pro Opening
        prompt = f"Topic: {state.topic}\n\nProvide a concise opening argument supporting this position."
        result: DebateMessageModel = mod_llm.invoke(f"{system}\n\n{prompt}")
        msg = DebateMessage(
            content=result.content,
            key_points=result.key_points,
            citations=[DebateCitation(**c.model_dump()) for c in result.citations]
        )
        state.history.append(DebateHistory(role="pro", phase="opening", output=msg))
    else:
        # Pro Rebuttal (respond to last turn)
        prior_text = state.history[-1].output.content
        prompt = (
            f"Topic: {state.topic}\n\n"
            f"Previous opposing argument: {prior_text}\n\n"
            f"Provide a concise rebuttal supporting your position."
        )
        result: DebateMessageModel = mod_llm.invoke(f"{system}\n\n{prompt}")
        msg = DebateMessage(
            content=result.content,
            key_points=result.key_points,
            citations=[DebateCitation(**c.model_dump()) for c in result.citations]
        )
        state.history.append(DebateHistory(role="pro", phase="rebuttal", output=msg))
    
    return state

def con_node(state: DebateState) -> DebateState:
    """Generate Con rebuttal or closing, then store into history."""
    system = (
        "You are the 'Con' debater in a formal debate. "
        "Oppose the topic with clear reasoning and evidence. "
        "Be concise and persuasive. Keep your response between 150-200 words. "
        # "Focus on 2-3 key points maximum. Provide 1-2 citations if relevant. "
    )
    mod_llm = llm.with_structured_output(schema=DebateMessageModel)

    # Safety: if somehow called first, do a rebuttal vs nothing
    prior_text = state.history[-1].output.content if state.history else ""

    # If the last phase was Pro opening -> Con rebuttal, else Con closing
    if state.history and state.history[-1].phase == "opening":
        prompt = (
            f"Topic: {state.topic}\n\n"
            f"Previous Pro argument: {prior_text}\n\n"
            f"Provide a concise rebuttal opposing this position."
        )
        phase = "rebuttal"
    else:
        prompt = (
            f"Topic: {state.topic}\n\n"
            f"Previous argument: {prior_text}\n\n"
            f"Provide a concise closing argument opposing this position."
        )
        phase = "closing"

    result: DebateMessageModel = mod_llm.invoke(f"{system}\n\n{prompt}")
    msg = DebateMessage(
        content=result.content,
        key_points=result.key_points,
        citations=[DebateCitation(**c.model_dump()) for c in result.citations]
    )
    state.history.append(DebateHistory(role="con", phase=phase, output=msg))
    return state

def judge_node(state: DebateState) -> DebateState:
    """Generate judge evaluation with strict length limits."""
    system = (
        "You are the Judge in a formal debate. "
        "Evaluate the arguments and provide a structured judgment. "
        "Be concise: rationale max 100 words, feedback max 75 words each. "
        "Focus on the strongest 2-3 key points that determined your decision."
    )
    mod_llm = llm.with_structured_output(schema=JudgeMessageModel)
    
    # Summarize debate history concisely to avoid token overflow
    debate_summary = []
    for h in state.history:
        debate_summary.append(f"{h.role.upper()} ({h.phase}): {h.output.content}")
    
    prompt = (
        f"Topic: {state.topic}\n\n"
        f"Debate Summary:\n" + "\n\n".join(debate_summary) + "\n\n"
        f"Provide your judgment."
    )
    
    result: JudgeMessageModel = mod_llm.invoke(f"{system}\n\n{prompt}")
    
    msg = JudgeMessage(
        decision=result.decision,
        rationale=result.rationale,
        pro_feedback=result.pro_feedback,
        con_feedback=result.con_feedback,
        key_points=result.key_points
    )
    state.history.append(DebateHistory(role="judge", phase="decision", output=msg))
    return state


# ---------- FLOW ROUTER ----------
def debate_flow_router(state: DebateState) -> str:
    """
    Turn order:
      - (no history) -> pro (opening)
      - after pro opening -> con (rebuttal)
      - after con rebuttal -> pro (rebuttal)
      - after pro rebuttal -> con (closing)
      - after con closing -> judge
      - after judge -> END
    """
    if not state.history:
        return "pro"

    last = state.history[-1]
    if last.role == "pro" and last.phase in ("opening", "rebuttal"):
        return "con"

    if last.role == "con":
        if last.phase == "rebuttal":
            return "pro"
        if last.phase == "closing":
            return "judge"
    
    if last.role == "judge":
        return "__end__"

    # Fallback
    return "__end__"


# -----------------------------
# 5) GRAPH WIRING
# -----------------------------
graph = StateGraph(DebateState)

# Existing nodes
graph.add_node("moderator", moderator_node)
graph.add_node("reject_exit", reject_exit)
graph.add_node("proceed", proceed_node)

# New debate nodes
graph.add_node("pro", pro_node)
graph.add_node("con", con_node)
graph.add_node("judge", judge_node)

graph.set_entry_point("moderator")

# Moderator decides whether to proceed or reject
graph.add_conditional_edges(
    "moderator",
    moderator_router,
    {"reject_exit": "reject_exit", "proceed": "proceed"}
)

graph.add_edge("reject_exit", END)

# Enter debate flow after proceed
graph.add_conditional_edges(
    "proceed",
    debate_flow_router,
    {"pro": "pro", "con": "con", "__end__": END}
)

# Keep routing between pro/con until closing -> END
graph.add_conditional_edges(
    "pro",
    debate_flow_router,
    {"pro": "pro", "con": "con", "__end__": END}
)
graph.add_conditional_edges(
    "con",
    debate_flow_router,
    {"pro": "pro", "con": "con", "judge": "judge", "__end__": END}
)
graph.add_conditional_edges(
    "judge",
    debate_flow_router,
    {"__end__": END}
)

app = graph.compile()

# -----------------------------
# 6) CLI entrypoint
# -----------------------------
if __name__ == "__main__":
    try:
        topic = input("Enter a debate topic: ")
    except KeyboardInterrupt:
        raise SystemExit

    initial = DebateState(topic=topic)

    print("\n--- Live Debate ---")
    final_state = None

    for event in app.stream(initial):  
        for node_name, payload in event.items():
            # In default mode, payload is a dict with metadata (value, logs, etc.)
            state = payload.get("value", payload)

            if node_name == "moderator":
                flags = state.get("safety_flags", [])
                last = flags[-1] if flags else {}
                action = last.get("action")
                print("\n[MODERATOR]")
                print("Topic:", state.get("topic"))
                if action == "reject":
                    print("Decision: REJECT")
                    print("Reason:", last.get("explanation", ""))
                elif action == "rephrase":
                    print("Decision: REPHRASE")
                    print("Safe Topic:", state.get("topic"))
                    print("Why:", last.get("explanation", ""))
                else:
                    print("Decision: ALLOW")

            elif node_name in ("pro", "con"):
                hlist = state.get("history", [])
                if not hlist:
                    continue
                h = hlist[-1]  # DebateHistory dataclass
                role = h.role.upper()
                phase = h.phase.capitalize()
                out = h.output  # DebateMessage dataclass
                print(f"\n[{role} - {phase}]")
                print(out.content)
                if out.key_points:
                    print("Key Points:", ", ".join(out.key_points))
                if out.citations:
                    print("Citations:")
                    for c in out.citations:
                        snip = f" - {c.snippet}" if c.snippet else ""
                        print(f" • {c.title} ({c.url}){snip}")

            elif node_name == "judge":
                hlist = state.get("history", [])
                if not hlist:
                    continue
                h = hlist[-1]
                out = h.output  # JudgeMessage dataclass
                print("\n[JUDGE DECISION]")
                print("Decision:", getattr(out, "decision", "").upper())
                print("Rationale:", getattr(out, "rationale", ""))
                pf = getattr(out, "pro_feedback", "")
                cf = getattr(out, "con_feedback", "")
                if pf:
                    print("Pro Feedback:", pf)
                if cf:
                    print("Con Feedback:", cf)
                kps = getattr(out, "key_points", []) or []
                if kps:
                    print("Key Points:", ", ".join(kps))

            final_state = state  # keep latest

    if final_state:
        print("\n--- Debate Complete ---")
        print("Final Topic:", final_state.get("topic"))


--- Live Debate ---

[MODERATOR]
Topic: ronaldo vs messi
Decision: ALLOW

[PRO - Opening]
Ladies and gentlemen, today I stand in favor of the argument that Cristiano Ronaldo is the superior player compared to Lionel Messi. While both athletes are undeniably exceptional, Ronaldo's versatility and physical prowess set him apart. 

Firstly, Ronaldo has proven himself in multiple leagues—Premier League, La Liga, Serie A—demonstrating his adaptability and skill across different styles of play. In contrast, Messi has spent the majority of his career at Barcelona, raising questions about his performance in varied environments.

Secondly, Ronaldo's goal-scoring record is unparalleled. He is the all-time leading scorer in the UEFA Champions League and has consistently outperformed in crucial matches, showcasing his ability to deliver under pressure. 

Moreover, Ronaldo's work ethic and dedication to fitness have allowed him to maintain peak performance well into his 30s, a testament to his pro